In [14]:
import tensorflow
import pandas as pd
import time
import numpy as np
import os
import matplotlib.pyplot as plt
import statistics

# tensorflow libraries
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import Callback

# sklearn libraries are useful for preprocessing, performance measures, etc.
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [15]:
path = 'output/'
dir = os.listdir(path)

In [43]:
df_combined = pd.DataFrame()

In [45]:
count = 0
for file in dir:
    if file[-4:] == '.csv':
        df_current_file = pd.read_csv(f'{path}\\{file}')
        number_of_rows = len(df_current_file.index)
        labels = [f'{file[:-4]}' for x in range(number_of_rows)]
        df_current_file['label'] = labels
        if count == 0:
            df_combined = df_current_file
        else:
            df_combined = pd.concat([df_combined, df_current_file])
        count += 1
df_combined.drop(axis=1, columns = ['file', 'start', 'end'])

['angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 

,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,pcm_intensity_sma_linregerrQ,...,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3,label
0,2.333378e-06,0.0,2.333378e-06,192.0,0.0,6.509605e-08,1.094024e-10,4.425491e-08,9.878091e-08,4.933604e-14,...,5.374218,2.671665,23.546236,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,angry
1,6.847944e-07,0.0,6.847944e-07,114.0,11.0,6.440258e-08,4.347192e-11,5.599076e-08,9.167785e-08,1.893454e-14,...,6.570551,3.170963,29.320164,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,angry
2,1.009743e-06,0.0,1.009743e-06,180.0,0.0,4.160852e-08,-5.480663e-11,5.103526e-08,5.940835e-08,1.285355e-14,...,5.214901,1.932646,12.729502,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,angry
3,2.720595e-06,0.0,2.720595e-06,192.0,0.0,9.149925e-08,-1.400926e-10,1.172062e-07,1.331957e-07,9.490879e-14,...,6.451539,4.864311,38.340054,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,angry
4,5.042887e-05,0.0,5.042887e-05,187.0,51.0,2.100998e-06,-4.468896e-10,2.191269e-06,3.026620e-06,3.048771e-11,...,7.231928,6.189695,52.337040,-0.791840,0.0,0.069879,0.791840,0.069879,0.861719,angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,5.456320e-07,0.0,5.456320e-07,181.0,0.0,3.433434e-08,-2.540327e-11,3.857668e-08,4.733004e-08,5.920616e-15,...,9.585387,2.926084,20.246042,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,surprised
188,1.278839e-06,0.0,1.278839e-06,187.0,0.0,8.063356e-08,3.198102e-11,7.444524e-08,1.174086e-07,3.673446e-14,...,6.818624,3.311348,24.112284,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,surprised
189,9.381138e-07,0.0,9.381138e-07,125.0,0.0,6.479801e-08,6.308280e-11,5.246532e-08,9.371777e-08,2.113311e-14,...,7.966315,3.445239,23.451675,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,surprised
190,1.825717e-06,0.0,1.825717e-06,122.0,0.0,1.319492e-07,-2.431059e-10,1.758298e-07,1.750552e-07,7.838204e-14,...,7.986842,4.458191,31.552225,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,surprised


In [ ]:
def prepare_dataset(df,  random_state):

    # Encode the labels from 0 to n_classes-1  
    label_encoder = preprocessing.LabelEncoder()
    df['label'] = label_encoder.fit_transform(df['label'])
  
    # devide data to train and test
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=random_state)
    
    # scale the training inputs
    x_train = df_train.drop(columns_to_drop,axis=1)
    y_train = df_train['label'].to_numpy()
    
    standard_scaler = preprocessing.StandardScaler()
    x_train_scaled = standard_scaler.fit_transform(x_train)

    #scale and prepare testing data
    x_test = df_test.drop(columns_to_drop,axis=1)
    x_test_scaled = standard_scaler.transform(x_test)
    y_test = df_test['label'].to_numpy() 
  
    return x_train_scaled, y_train, x_test_scaled, y_test

In [ ]:
with tf.device('cpu:0'):
  embedding_layer = layers.Embedding(...)
  embedding_layer.build()